In [ ]:
#bibliotecas 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def decompoe_forca(fa, fb, a, b):



def calcula_resultante(f, a, b):
    fa = f(a)
    fb = f(b)
    resultante = 0

    # força constante, pode ser 0 ou retangular
    if fa == fb: 
        if fa == 0:      
            return resultante, 0
        else:            
            resultante, x = (b-a)*fa, (a+b)/2

    # força triangular crescente
    if fa == 0 and fb > 0: 
        resultante, x = (b-a)*fb/2, (b-a) * 2/3 
    
    # força triangular decrescente
    if fa > 0 and fb == 0:
        resultante, x = (b-a)*fa/2, (b-a) * 1/3

    if fa > 0 and fb > 0 and fa != fb:
        resultante, resultante2 = decompoe_forca(fa, fb, a, b) 
    return resultante, x